# Association rule mining with Apriori algorithm

In [5]:
import numpy as np
import pandas as pd
import re
from efficient_apriori import apriori
import time
import datetime

In [6]:
size = 50

## Data preparation

In [7]:
colnames = ["class_name","handicapped_infants","water_project_cost_sharing","adoption_of_the_budget_resolution",
            "physician_fee_freeze","el_salvador_aid","religious_groups_in_schools","anti_satellite_test_ban",
            "aid_to_nicaraguan_contras","mx_missile","immigration","synfuels_corporation_cutback",
            "education_spending","superfund_right_to_sue","crime","duty_free_exports",
            "export_administration_act_south_africa"]

data = pd.read_csv("/home/laura/Scrivania/KDMproj/house-votes-84.txt", sep = ',',na_values = '?', header = None, 
                   names = colnames)
data = pd.DataFrame(data)

data["republican"] = 0
data.loc[data.class_name=="republican","republican"] = pd.Series(np.ones(len(data.class_name=="republican"),dtype=int))
del data["class_name"]
del colnames[0]

data.head()

,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa,republican
0,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y,1
1,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN,1
2,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n,0
3,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y,0
4,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y,0


In [8]:
data = data.iloc[:size,]

In [9]:
transactions = []
for i in range(len(data)):
    t = []
    for j in range(len(data.columns)-1):
        if data.iloc[i,j] == "y":
            t.append(data.columns[j]+"_yes")
        if data.iloc[i,j] == "n":
            t.append(data.columns[j]+"_no")
    if data.iloc[i,16] == 1:
        t.append("republican")
    if data.iloc[i,16] == 0:
        t.append("democrat")
    transactions.append(t)

## Association rule extraction

In [10]:
# Start point
start_time = time.time()
print("Start time: ", datetime.datetime.now())

Start time:  2019-09-27 17:06:02.108024


In [11]:
itemset, rules = apriori(transactions, min_support=0.2, min_confidence=1)

In [12]:
clean_rules = []
for rule in rules:
    s=str(rule)
    s=re.sub(" \(.*", "", s)
    s=re.sub("{", "", s)
    s=re.sub("}", "", s)
    s=s.split(" -> ")
    for i, sub in enumerate(s):
        s[i] = set(sub.split(", "))
    clean_rules.append(s)

In [13]:
print("\n{} rules found \n".format(len(clean_rules)))


576193 rules found 



In [14]:
min_cons = []
for rule in clean_rules:
    if len(rule[1]) == 1:
        min_cons.append(rule)

min_pred = []
min_rules = []
for rule in min_cons:
    is_minimal = True
    for p in min_pred:
        if p < rule[0]:
            is_minimal = False
    if is_minimal:
        min_pred.append(rule[0])
        min_rules.append(rule)

print("\n{} minimal rules found \n".format(len(min_rules)))


161 minimal rules found 



In [37]:
#min_rules

In [15]:
looking_for = "physician_fee_freeze_yes"

for rule in min_rules:
    if looking_for in rule[0] or looking_for in rule[1]:
        print(rule)

[{'physician_fee_freeze_yes'}, {'adoption_of_the_budget_resolution_no'}]
[{'adoption_of_the_budget_resolution_no'}, {'physician_fee_freeze_yes'}]
[{'physician_fee_freeze_yes'}, {'el_salvador_aid_yes'}]
[{'republican'}, {'physician_fee_freeze_yes'}]
[{'duty_free_exports_no', 'el_salvador_aid_yes', 'synfuels_corporation_cutback_no'}, {'physician_fee_freeze_yes'}]
[{'duty_free_exports_no', 'superfund_right_to_sue_yes', 'synfuels_corporation_cutback_no'}, {'physician_fee_freeze_yes'}]


In [39]:
# End point
end_time = time.time()
uptime = end_time - start_time
human_uptime = datetime.timedelta(seconds=uptime)

print("End time: ", datetime.datetime.now())
print("Uptime :" ,human_uptime)

End time:  2019-09-27 10:49:56.671591
Uptime : 0:00:25.042443
